<a href="https://colab.research.google.com/github/kolayn808/textattack_test/blob/main/Textattack_on_TensorFlow_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Установка Textattack

In [1]:
!git clone https://github.com/QData/TextAttack.git


Cloning into 'TextAttack'...
remote: Enumerating objects: 21081, done.
remote: Counting objects: 100% (546/546), done.
remote: Compressing objects: 100% (290/290), done.
remote: Total 21081 (delta 314), reused 403 (delta 253), pack-reused 20535
Receiving objects: 100% (21081/21081), 25.85 MiB | 29.54 MiB/s, done.
Resolving deltas: 100% (15619/15619), done.


In [2]:
%cd /content/TextAttack
!pip install -e .

/content/TextAttack
Obtaining file:///content/TextAttack
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 49.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11

Импорт модели

In [3]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"],
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Version:  2.13.0
Eager mode:  True
Hub version:  0.15.0
GPU is available


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete91RQDT/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete91RQDT/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/40
30/30 [==============================] - 14s 280ms/step - loss: 0.6787 - accuracy: 0.5396 - val_loss: 0.6468 - val_accuracy: 0.56

In [4]:
import numpy as np
import torch

from textattack.models.wrappers import ModelWrapper

class CustomTensorFlowModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
        text_array = np.array(text_input_list)
        preds = self.model(text_array).numpy()
        logits = torch.exp(-torch.tensor(preds))
        logits = 1 / (1 + logits)
        logits = logits.squeeze(dim=-1)
        # Since this model only has a single output (between 0 or 1),
        # we have to add the second dimension.
        final_preds = torch.stack((1-logits, logits), dim=1)
        return final_preds

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
CustomTensorFlowModelWrapper(model)(['I hate you so much', 'I love you'])

tensor([[0.2203, 0.7797],
        [0.0101, 0.9899]])

In [6]:
model_wrapper = CustomTensorFlowModelWrapper(model)

from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PWWSRen2019
from textattack import Attacker

dataset = HuggingFaceDataset("rotten_tomatoes", None, "test", shuffle=True)
attack = PWWSRen2019.build(model_wrapper)

attacker = Attacker(attack, dataset)
attacker.attack_dataset()

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

textattack: Loading datasets dataset rotten_tomatoes, split test.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
textattack: Unknown if model of class <class 'keras.src.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



...

[[Positive (52%)]] --> [[Negative (95%)]]

consistently clever and [[suspenseful]] .

consistently clever and [[cliff-hanging]] .




[Succeeded / Failed / Skipped / Total] 3 / 0 / 3 / 6:  60%|██████    | 6/10 [00:00<00:00,  6.72it/s]

--------------------------------------------- Result 3 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (69%)]]

it's like a " [[big]] chill " reunion of the baader-meinhof gang , only these [[guys]] are more harmless pranksters than political activists .

it's like a " [[bad]] chill " reunion of the baader-meinhof gang , only these [[cat]] are more harmless pranksters than political activists .


--------------------------------------------- Result 4 ---------------------------------------------
[[Negative (75%)]] --> [[[SKIPPED]]]

the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .


--------------------------------------------- Result 5 ---------------------------------------------
[[Negative (95%)]] --> [[[SKIPPED]]]

red dragon " never cuts corners .


--------------------------------------------- Result 6 ---------------------------------------------
[[Positive (100%)

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10: 100%|██████████| 10/10 [00:01<00:00,  8.35it/s]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (91%)]]

[[throws]] in enough clever and unexpected [[twists]] to make the formula feel fresh .

[[drop]] in enough clever and unexpected [[device]] to make the formula feel fresh .


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (95%)]] --> [[Negative (70%)]]

weighty and ponderous but every [[bit]] as filling as the treat of the title .

weighty and ponderous but every [[bite]] as filling as the treat of the title .


--------------------------------------------- Result 9 ---------------------------------------------
[[Positive (82%)]] --> [[Negative (66%)]]

a [[real]] audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .

a [[material]] audience-pleaser that will strike a chord with a